In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Step 1: Simulate observed trades from another venue
np.random.seed(42)
trade_data = pd.DataFrame({
    "timestamp": pd.date_range(start="2024-01-01 09:30:00", periods=100, freq="T"),
    "price": np.cumsum(np.random.normal(0, 1, 100)) + 100,  # Random walk around 100
    "volume": np.random.randint(1, 100, 100)
})

# Step 2: Define a strategy to detect directional changes
def detect_signal(data, threshold=0.5):
    data["price_change"] = data["price"].diff()
    data["signal"] = np.where(data["price_change"] > threshold, "buy",
                              np.where(data["price_change"] < -threshold, "sell", "hold"))
    return data

# Step 3: Simulate taking trades on IEX
def execute_trades(data, initial_cash=10000):
    cash = initial_cash
    position = 0  # Number of shares held
    trade_log = []

    for _, row in data.iterrows():
        if row["signal"] == "buy":
            position += 1
            cash -= row["price"]  # Removing liquidity means paying market price
            trade_log.append((row["timestamp"], "buy", row["price"], cash, position))
        elif row["signal"] == "sell" and position > 0:
            position -= 1
            cash += row["price"]
            trade_log.append((row["timestamp"], "sell", row["price"], cash, position))
        else:
            trade_log.append((row["timestamp"], "hold", row["price"], cash, position))

    trade_df = pd.DataFrame(trade_log, columns=["timestamp", "action", "price", "cash", "position"])
    return trade_df, cash + position * data.iloc[-1]["price"]

# Execute the strategy
trade_data = detect_signal(trade_data)
trade_log, final_portfolio_value = execute_trades(trade_data)

# Output results
print("Final Portfolio Value:", final_portfolio_value)
print(trade_log.head())

Final Portfolio Value: 9962.306833760893
            timestamp action       price          cash  position
0 2024-01-01 09:30:00   hold  100.496714  10000.000000         0
1 2024-01-01 09:31:00   hold  100.358450  10000.000000         0
2 2024-01-01 09:32:00    buy  101.006138   9898.993862         1
3 2024-01-01 09:33:00    buy  102.529168   9796.464693         2
4 2024-01-01 09:34:00   hold  102.295015   9796.464693         2


<ipython-input-2-3a17f72c0577>:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  "timestamp": pd.date_range(start="2024-01-01 09:30:00", periods=100, freq="T"),
